# 주거상담기록 텍스트 전처리

## 1. 텍스트 정제

### 1.1. 패지키 설치 및 데이터 불러오기

In [ ]:
#!pip install pandas
#!pip install tqdm
#!pip install kiwipiepy

In [2]:
## 필요한 모듈 불러오기
import json, random, re, os
import swifter
import pandas as pd
from tqdm import tqdm # 작업 프로세스 시각화
from kiwipiepy import Kiwi # 형태소분석기 모듈
from kiwipiepy.utils import Stopwords # 불용어사전 담겨있는 모듈 불러오기

In [3]:
print(os.getcwd()) # 현재 디렉토리 경로 확인하기
db_record = pd.read_csv('D:/housingRecordAnalytics/Data/db_record_v1_230106.csv')[["id_f", "cons_text"]] # 심층상담 데이터 읽기

\\jsk\server 1\Project\Ongoing\housingConsultRecord\2-Work\jhyeonpark\Code


C:\Users\UNIST\AppData\Local\Temp/ipykernel_32124/1285594222.py:2: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  db_record = pd.read_csv('D:/housingRecordAnalytics/Data/db_record_v1_230106.csv')[["id_f", "cons_text"]] # 심층상담 데이터 읽기


### 1.2. 불필요한 텍스트 제거

In [4]:
# db_record = db_record.sample(1000)

In [5]:
## 한글, 숫자만 남기고 제거; 줄바꿈
db_record['cons_text_cleaned_1'] = list(map(lambda text: re.sub(r"([^가-힣0-9a-z ])", " ", text), db_record['cons_text'].str.lower()))

### 1.3. 동의어 사전 적용

In [6]:
dict_synonyms = pd.read_csv('../data/dict/dict_synonyms.csv')

## 동의어사전 적용하는 함수 정의
def replace_word(text):
    for i in range(len(dict_synonyms['beforeWord'])): 
        try:
            if dict_synonyms['beforeWord'][i] in text: # 바꿀 단어가 발견된다면
                text = text.replace(dict_synonyms['beforeWord'][i], dict_synonyms['afterWord'][i]) # 해당하는 단어의 afterWord로 바꾸어라
        except Exception as e:
            print(f"Error 발생 / 에러명: {e}")
            print(dict_synonyms['afterWord'][i])
            print(text)
    return text

db_record['cons_text_cleaned_2'] = db_record['cons_text_cleaned_1'].swifter.set_npartitions(npartitions = 12).apply(lambda x: replace_word(x))

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

## 2. 텍스트 토큰화

### 2.1. 사용자 정의 사전 만들기

In [9]:
## 동의어사전에서 추출하기
dict_synonyms = pd.read_csv('../data/dict/dict_synonyms.csv') # 동의어사전 불러오기
list_userDefined_synonyms = (dict_synonyms['afterWord'].str.strip().unique() + "\tNNG\t9").tolist() # 사용자 정의 사전 포맷으로 변경

## 사용자 정의 사전 불러오기
dict_add = pd.read_table("../Data/dict/dict_userDefined_madeByMe.txt") # 추가 사전 불러오기
list_userDefined_add = (pd.read_table("../Data/dict/dict_userDefined_madeByMe.txt", header = None, sep = "-")[0] + "\t9").tolist() # 사용자 정의 사전 포맷으로 변경

## 위 두 사전 합치기
dict_userDefined = pd.DataFrame(pd.DataFrame(list_userDefined_synonyms + list_userDefined_add)[0].unique()) # 두 list 결합
dict_userDefined.to_csv('../Data/dict/dict_userDefined.txt', index = False, header = False) # 새로운 사용자 정의 사전 저장하기

### 2.2. 형태소분석기

In [ ]:
## 설정
# kiwi = Kiwi(num_workers = 0, model_type = 'sbg')
# kiwi.load_user_dictionary('../Data/dict/dict_userDefined.txt') # 사용자사전 추가
# kiwi.tokenize('당센터 상담 후 임대주택 입주 대상자 사례관리 물품 전달을 위해 연락 드림. -12월 17일 9시에 내방해주신다고 함.')

In [ ]:
## Kiwi 설정
kiwi = Kiwi(model_type = 'sbg', typos = 'basic') # Kiwi 추가 설정
kiwi.load_user_dictionary('../Data/dict/dict_userDefined.txt') # 사용자사전 추가
morph_analysis = lambda x: kiwi.tokenize(x, stopwords = Stopwords()) if type(x) is str else None # 형태소 분석
db_record['cons_text_posTagged_kiwi'] = db_record['cons_text_cleaned_2'].swifter.apply(morph_analysis)

### 2.3. 사용자/불용어 사전

In [ ]:
## 형태소분석된 것들을 join
dict_stopword = pd.read_csv('../data/dict/dict_stopword.csv')
dict_oneChar = pd.read_csv("../Data/dict/dict_oneChar.csv")

## 품사 정의하기
list_pos_all = ("N", "V", "M", "J", "E", "X") # 주요 품사 모두
list_pos_main = ("N", "V") # 명사, 동사, 형용사만

## 불용어 제거하고 형태소 결합하기
def tokenToText(tokens):
    textJoinedMain = []
    textJoinedAll = [] 
    for token, pos, _, _ in tokens:        
        if (token not in list(dict_stopword['word']) and ( len(token) > 1 or (len(token) == 1 and token in list(dict_oneChar['word']) ) )): 
            if pos.startswith('V'): token = token + '다' # 동사라면 뒤에 '다'를 붙여서 자연스럽게 만들기
            if pos.startswith(list_pos_main) : textJoinedMain.append(token)
            if pos.startswith(list_pos_all) : textJoinedAll.append(token)

    out = pd.Series([' '.join(textJoinedAll), ' '.join(textJoinedMain)])
    return out

db_record[['cons_text_posTagged_all', 'cons_text_posTagged_main']] = db_record['cons_text_posTagged_kiwi'].swifter.set_npartitions(npartitions = 6).apply(lambda x: tokenToText(x))

## 3. 데이터 내보내기

In [ ]:
db_record = db_record[['id_f', 'cons_text', 'cons_text_posTagged_main']]

In [ ]:
db_record.sample(frac = 0.005).to_csv('D:/housingRecordAnalytics/Data/db_record_v2_sample.csv', index = False, encoding="utf-8-sig") # 1% 샘플데이터
db_record.to_csv('D:/housingRecordAnalytics/Data/db_record_v2_' + pd.to_datetime("today").strftime("M%mD%d_H%HM%M") + '.csv', index=False, encoding="utf-8-sig") # 본 데이터